## **Using LLMs for Prompt Engineering**

The core idea here is to prompt the agent to reflect on its own performance and modify its own instructions.

In [1]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

In [2]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], template=template
    )

    chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=prompt,
        verbose=True,
        memory=ConversationBufferWindowMemory(),
    )
    return chain


def initialize_meta_chain():
    meta_template = """
    Assistant has just had the below interactions with a User.
    Assistant followed their "Instructions" closely.
    Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance.
    What could Assistant have done better?
    What should the Assistant remember about this user?
    Are there things this user always wants?
    Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future.
    Assistant's goal is to satisfy the user in as few interactions as possible.
    Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions.
    Don't forget any important details in the current Instructions!
    Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], template=meta_template
    )

    meta_chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=meta_prompt,
        verbose=True,
    )
    return meta_chain


def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history


def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter) :]
    return new_instructions

In [7]:
def main(task, max_iters=3, max_meta_iters=1):
    failed_phrase = "task failed"
    success_phrase = "task succeeded"
    key_phrases = [success_phrase, failed_phrase]

    instructions = "None"
    for i in range(max_meta_iters):
        print(f"[Episode {i+1}/{max_meta_iters}]")
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f"(Step {j+1}/{max_iters})")
            print(f"Assistant: {output}")
            print("Human: ")
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print("You succeeded! Thanks for playing!")
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f"Feedback: {meta_output}")
        instructions = get_new_instructions(meta_output)
        print(f"New Instructions: {instructions}")
        print("\n" + "#" * 80 + "\n")
    print("You failed! Thanks for playing!")

In [8]:
task = "Provide a coherent argument as to why companies should spend ensuring their Data teams all code in Python & SQL only."

In [9]:
task = "Provide a coherent argument as to why companies should spend ensuring their Data teams all code in Python & SQL only."
main(task=task, max_iters=3, max_meta_iters=3)

[Episode 1/3]


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    
    Human: Provide a coherent argument as to why companies should spend ensuring their Data teams all code in Python & SQL only.
    Assistant:

> Finished chain.
(Step 1/3)
Assistant:  Companies should invest in ensuring their Data teams code in Python & SQL only because these two languages are the most popular and widely used programming languages for data analysis. Python is a powerful language that is easy to learn and use, and it is used for a variety of tasks such as data cleaning, data manipulation, and data visualization. SQL is a powerful language for querying and manipulating data, and it is used for tasks such as data analysis, data mining, and data warehousing. Both languages are essential for data analysis, and they are the most popular and widely used languages for data analysis. By investing in ensuring their Data teams code in Python & SQL only, companies can ensure th

In [10]:
class LLMChainInitializer:
    def __init__(self, llm_temperature=0, verbose=True):
        self.llm_temperature = llm_temperature
        self.verbose = verbose

    def initialize_chain(self, instructions, memory=None):
        if memory is None:
            memory = ConversationBufferWindowMemory()
            memory.ai_prefix = "Assistant"

        prompt_template = self._create_prompt_template(instructions, memory.memory_key)
        chain = LLMChain(
            llm=OpenAI(temperature=self.llm_temperature),
            prompt=prompt_template,
            verbose=self.verbose,
            memory=ConversationBufferWindowMemory()  # Consider reusing 'memory' here if appropriate
        )
        return chain

    def initialize_meta_chain(self):
        meta_template = self._get_meta_template()
        meta_prompt = PromptTemplate(
            input_variables=["chat_history"], template=meta_template
        )

        meta_chain = LLMChain(
            llm=OpenAI(temperature=self.llm_temperature),
            prompt=meta_prompt,
            verbose=self.verbose,
        )
        return meta_chain

    @staticmethod
    def _create_prompt_template(instructions, memory_key):
        template = f"""
        Instructions: {instructions}
        {{{{memory_key}}}}
        Human: {{human_input}}
        Assistant:"""
        return PromptTemplate(
            input_variables=["history", "human_input"], template=template
        )

    @staticmethod
    def _get_meta_template():
        return """
    Assistant has just had the below interactions with a User.
    Assistant followed their "Instructions" closely.
    Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance.
    What could Assistant have done better?
    What should the Assistant remember about this user?
    Are there things this user always wants?
    Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future.
    Assistant's goal is to satisfy the user in as few interactions as possible.
    Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions.
    Don't forget any important details in the current Instructions!
    Indicate the new Instructions by "Instructions: ...".
    """

def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history

def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter):]
    return new_instructions
